# 🔧 Custom Build Motorcycles - Ithaka Powertrain Simulator



In [ ]:
#@title 🚀 Environment Setup { display-mode: "form" }

#@markdown This cell automatically detects your environment and sets up all required dependencies.
#@markdown **Click the play button** to run this cell - setup takes about 30-60 seconds.

import sys
import os
import warnings
import subprocess
warnings.filterwarnings('ignore')

def setup_environment():
    """Smart environment setup for both Colab and local environments."""

    # Detect environment
    try:
        import google.colab
        IN_COLAB = True
        print("Google Colab environment detected")
    except ImportError:
        IN_COLAB = False
        print("Local environment detected")

    if IN_COLAB:
        # Enable widgets in Colab
        try:
            from google.colab import output
            output.enable_custom_widget_manager()
        except:
            pass
            
        # Check for existing repository
        print("Checking for repository...")

        current_dir = os.getcwd()
        possible_paths = ['/content/Teo-Moto-Repo', '/content/ithaka-powertrain-sim', current_dir]

        repo_found = False
        for path in possible_paths:
            if os.path.exists(path) and os.path.exists(os.path.join(path, 'setup.py')):
                print(f"Found repository at: {path}")
                os.chdir(path)
                repo_found = True
                break

        if not repo_found:
            print("Cloning repository...")
            try:
                result = subprocess.run(['git', 'clone', 'https://github.com/Teodus/Teo-Moto-Repo.git'],
                                      capture_output=True, text=True, cwd='/content')
                if result.returncode == 0:
                    os.chdir('/content/Teo-Moto-Repo')
                    print("Repository cloned successfully")
                else:
                    print("Clone failed. Please check your internet connection.")
                    return False
            except Exception as e:
                print(f"Error: {e}")
                return False

        # Install dependencies
        print("Installing dependencies...")
        subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', '-r', 'requirements.txt'],
                      capture_output=True)
        subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', '-e', '.'],
                      capture_output=True)
        print("Dependencies installed")

    # Verify installation
    try:
        print("Verifying installation...")
        import pandas as pd
        import numpy as np
        import matplotlib.pyplot as plt
        import ipywidgets as widgets
        from IPython.display import display, clear_output, HTML
        import json
        import glob
        from datetime import datetime

        # Import component builders
        from ithaka_powertrain_sim.component_library import (
            Battery_10kWh_200WhKg, Battery_15kWh_220WhKg, Battery_20kWh_180WhKg, Battery_25kWh_200WhKg,
            Motor_30kW_MidDrive, Motor_50kW_HighPerf, Motor_80kW_HighPerf, Motor_120kW_HighPerf,
            Engine_400cc_30kW, Engine_500cc_40kW, Engine_650cc_50kW, Engine_750cc_60kW, Engine_1000cc_80kW,
            FuelTank_15L, FuelTank_25L
        )
        from ithaka_powertrain_sim.motorbike import Motorbike
        from ithaka_powertrain_sim.trajectory import load_gpx, append_and_resample_dataframe

        print("All modules imported successfully")
        print("Component builders loaded and ready")

        gpx_files = glob.glob('docs/gpx_files/*.gpx')
        if gpx_files:
            print(f"Found {len(gpx_files)} test tracks")

        return True

    except ImportError as e:
        print(f"Import failed: {e}")
        if 'shortuuid' in str(e):
            print("Installing missing dependency...")
            subprocess.run([sys.executable, '-m', 'pip', 'install', 'shortuuid'],
                         capture_output=True)
            print("Please re-run this cell")
        return False

# Run setup
print("Starting environment setup...")
setup_success = setup_environment()

if setup_success:
    print("\nSetup complete! Ready to build custom motorcycles!")
    print("Continue to Step 1 below")
else:
    print("\nSetup failed. Please check the error messages above.")

In [ ]:
#@title 🏗️ Initialize Custom Builder { display-mode: "form" }

#@markdown This cell sets up the custom motorcycle builder with all component options.
#@markdown **Run this cell** to prepare the design environment.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import json
import os
from datetime import datetime

# Import component builders
from ithaka_powertrain_sim.component_library import (
    Battery_10kWh_200WhKg, Battery_15kWh_220WhKg, Battery_20kWh_180WhKg, Battery_25kWh_200WhKg,
    Motor_30kW_MidDrive, Motor_50kW_HighPerf, Motor_80kW_HighPerf, Motor_120kW_HighPerf,
    Engine_400cc_30kW, Engine_500cc_40kW, Engine_650cc_50kW, Engine_750cc_60kW, Engine_1000cc_80kW,
    FuelTank_15L, FuelTank_25L
)
from ithaka_powertrain_sim.motorbike import Motorbike
from ithaka_powertrain_sim.trajectory import load_gpx, append_and_resample_dataframe

# Import simulation and utility modules
from ithaka_powertrain_sim.simulation import SimulationRunner
from ithaka_powertrain_sim.component_library.notebook_utils import TrackManager, SimulationExporter, UIComponents
from ithaka_powertrain_sim.component_library.interactive_picker import ensure_widget_display

# Ensure widgets display properly in Colab
ensure_widget_display()

class CustomMotorcycleBuilder:
    """Application state manager for custom motorcycle building."""

    def __init__(self):
        self.custom_motorcycle = None
        self.selected_tracks = []
        self.simulation_results = {}
        self.selected_motor = None
        self.selected_battery = None
        self.selected_engine = None
        self.selected_fuel_tank = None
        self.motorcycle_type = 'Pure EV'
        
        # Initialize simulation runner with optimized settings
        self.sim_runner = SimulationRunner(
            min_speed_kmh=10.0,  # Consistent minimum speed
            max_speed_kmh=300.0,  # Realistic maximum
            use_filter=True,      # Filter stops
            speed_calculation_method='distance_time'  # Use proper average
        )
        
        # Track manager for handling track files
        self.track_manager = TrackManager()

    def get_available_tracks(self):
        """Get list of available GPX track files."""
        return self.track_manager.get_available_tracks()

    def build_motorcycle(self):
        """Build the custom motorcycle from selected components."""
        components = []

        # Add selected components
        if self.selected_motor:
            if '30kW' in self.selected_motor:
                components.append(Motor_30kW_MidDrive())
            elif '50kW' in self.selected_motor:
                components.append(Motor_50kW_HighPerf())
            elif '80kW' in self.selected_motor:
                components.append(Motor_80kW_HighPerf())
            elif '120kW' in self.selected_motor:
                components.append(Motor_120kW_HighPerf())

        if self.selected_battery:
            if '10kWh' in self.selected_battery:
                components.append(Battery_10kWh_200WhKg())
            elif '15kWh' in self.selected_battery:
                components.append(Battery_15kWh_220WhKg())
            elif '20kWh' in self.selected_battery:
                components.append(Battery_20kWh_180WhKg())
            elif '25kWh' in self.selected_battery:
                components.append(Battery_25kWh_200WhKg())

        if self.selected_engine:
            if '400cc' in self.selected_engine:
                components.append(Engine_400cc_30kW())
            elif '500cc' in self.selected_engine:
                components.append(Engine_500cc_40kW())
            elif '650cc' in self.selected_engine:
                components.append(Engine_650cc_50kW())
            elif '750cc' in self.selected_engine:
                components.append(Engine_750cc_60kW())
            elif '1000cc' in self.selected_engine:
                components.append(Engine_1000cc_80kW())

        if self.selected_fuel_tank:
            if '15L' in self.selected_fuel_tank:
                components.append(FuelTank_15L())
            elif '25L' in self.selected_fuel_tank:
                components.append(FuelTank_25L())

        if not components:
            raise ValueError("No components selected. Please select at least one component.")

        # Create motorcycle name
        timestamp = datetime.now().strftime('%H%M%S')
        name = f"Custom_{self.motorcycle_type.replace(' ', '_')}_{timestamp}"

        # Create motorcycle with correct parameters
        self.custom_motorcycle = Motorbike(
            name=name,
            child_components=components,
            dry_mass_excluding_components=150.0,
            front_mass_ratio=0.4,
            front_wheel_inertia=0.8,
            front_wheel_radius=0.3,
            front_wheel_coefficient_of_rolling_resistance=0.015,
            rear_wheel_inertia=1.2,
            rear_wheel_radius=0.3,
            rear_wheel_coefficient_of_rolling_resistance=0.015,
            frontal_area=0.6,
            coefficient_of_aerodynamic_drag=0.6
        )

        return self.custom_motorcycle

    def simulate_motorcycle_on_track(self, motorcycle, track_file, track_name):
        """Run simulation for motorcycle on single track using SimulationRunner."""
        return self.sim_runner.simulate_motorcycle_on_track(
            motorcycle,
            track_file,
            track_name,
            verbose=False
        )

# Initialize application
app = CustomMotorcycleBuilder()

# Define widget styles
style = {'description_width': '180px'}
layout_wide = widgets.Layout(width='500px')
layout_medium = widgets.Layout(width='400px')

print("✅ Custom Motorcycle Builder initialized successfully!")
print("🚀 Enhanced simulation features active:")
print("  - Consistent 10 km/h minimum speed filtering")
print("  - Proper distance/time average speed calculation")
print("  - Improved error handling and validation")
print("📍 Continue to Step 1 below")

## Step 1: Design Your Custom Motorcycle

In [ ]:
#@title 🔧 Custom Motorcycle Designer { display-mode: "form" }

#@markdown Design your custom motorcycle by selecting the type and components.
#@markdown **Real-time feedback** shows weight, power, and performance characteristics as you build.

# Step 1: Motorcycle Type Selection
print("STEP 1: Choose Your Powertrain Type")
print("=" * 50)

type_selector = widgets.RadioButtons(
    options=[
        ('Pure EV - Electric Only', 'Pure EV'),
        ('Pure ICE - Internal Combustion Only', 'Pure ICE'),
        ('Hybrid - Electric + ICE Combination', 'Hybrid')
    ],
    value='Pure EV',
    description='Powertrain Type:',
    style=style,
    layout=layout_wide
)

display(type_selector)

# Component selectors
print("\nSTEP 2: Select Components")
print("=" * 50)

# Electric Motor Selection
motor_options = [('Select motor...', None)]
motor_options.extend([
    ('30kW Mid-Drive Motor (30kW, 28kg)', '30kW Mid-Drive Motor'),
    ('50kW High-Performance Motor (50kW, 35kg)', '50kW High-Performance Motor'),
    ('80kW High-Performance Motor (80kW, 45kg)', '80kW High-Performance Motor'),
    ('120kW High-Performance Motor (120kW, 60kg)', '120kW High-Performance Motor')
])

motor_selector = widgets.Dropdown(
    options=motor_options,
    value=None,
    description='Electric Motor:',
    style=style,
    layout=layout_wide
)

# Battery Selection
battery_options = [('Select battery...', None)]
battery_options.extend([
    ('10kWh Battery (10kWh, 62kg, 200Wh/kg)', '10kWh Battery'),
    ('15kWh Battery (15kWh, 68kg, 220Wh/kg)', '15kWh Battery'),
    ('20kWh Battery (20kWh, 111kg, 180Wh/kg)', '20kWh Battery'),
    ('25kWh Battery (25kWh, 125kg, 200Wh/kg)', '25kWh Battery')
])

battery_selector = widgets.Dropdown(
    options=battery_options,
    value=None,
    description='Battery Pack:',
    style=style,
    layout=layout_wide
)

# Engine Selection
engine_options = [('Select engine...', None)]
engine_options.extend([
    ('400cc Engine (400cc, 30kW, 45kg)', '400cc Engine'),
    ('500cc Engine (500cc, 40kW, 55kg)', '500cc Engine'),
    ('650cc Engine (650cc, 50kW, 65kg)', '650cc Engine'),
    ('750cc Engine (750cc, 60kW, 75kg)', '750cc Engine'),
    ('1000cc Engine (1000cc, 80kW, 90kg)', '1000cc Engine')
])

engine_selector = widgets.Dropdown(
    options=engine_options,
    value=None,
    description='ICE Engine:',
    style=style,
    layout=layout_wide
)

# Fuel Tank Selection
fuel_options = [('Select fuel tank...', None)]
fuel_options.extend([
    ('15L Fuel Tank (15L, 12kg)', '15L Fuel Tank'),
    ('25L Fuel Tank (25L, 20kg)', '25L Fuel Tank')
])

fuel_selector = widgets.Dropdown(
    options=fuel_options,
    value=None,
    description='Fuel Tank:',
    style=style,
    layout=layout_wide
)

# Create containers for component groups
electric_components = widgets.VBox([
    widgets.HTML("<h4>Electric Components</h4>"),
    motor_selector,
    battery_selector
])

ice_components = widgets.VBox([
    widgets.HTML("<h4>Internal Combustion Components</h4>"),
    engine_selector,
    fuel_selector
])

# Real-time specifications display
specs_display = widgets.Output()

# Build button
build_btn = widgets.Button(
    description='Build My Motorcycle',
    button_style='success',
    layout=widgets.Layout(width='200px', height='40px')
)

build_output = widgets.Output()

def update_component_visibility(change=None):
    """Update which components are visible based on motorcycle type."""
    app.motorcycle_type = type_selector.value

    if app.motorcycle_type == "Pure EV":
        electric_components.layout.display = 'block'
        ice_components.layout.display = 'none'
    elif app.motorcycle_type == "Pure ICE":
        electric_components.layout.display = 'none'
        ice_components.layout.display = 'block'
    elif app.motorcycle_type == "Hybrid":
        electric_components.layout.display = 'block'
        ice_components.layout.display = 'block'

    update_specifications()

def handle_motor_change(change):
    app.selected_motor = change.new
    update_specifications()

def handle_battery_change(change):
    app.selected_battery = change.new
    update_specifications()

def handle_engine_change(change):
    app.selected_engine = change.new
    update_specifications()

def handle_fuel_change(change):
    app.selected_fuel_tank = change.new
    update_specifications()

def update_specifications(change=None):
    """Update real-time motorcycle specifications."""
    with specs_display:
        clear_output()

        # Calculate specifications
        base_mass = 150.0  # kg
        total_mass = base_mass
        total_power = 0
        total_energy = 0
        components = []

        if app.selected_motor:
            components.append(f"Motor: {app.selected_motor}")
            if '30kW' in app.selected_motor:
                total_mass += 28
                total_power += 30
            elif '50kW' in app.selected_motor:
                total_mass += 35
                total_power += 50
            elif '80kW' in app.selected_motor:
                total_mass += 45
                total_power += 80
            elif '120kW' in app.selected_motor:
                total_mass += 60
                total_power += 120

        if app.selected_battery:
            components.append(f"Battery: {app.selected_battery}")
            if '10kWh' in app.selected_battery:
                total_mass += 62
                total_energy += 10
            elif '15kWh' in app.selected_battery:
                total_mass += 68
                total_energy += 15
            elif '20kWh' in app.selected_battery:
                total_mass += 111
                total_energy += 20
            elif '25kWh' in app.selected_battery:
                total_mass += 125
                total_energy += 25

        if app.selected_engine:
            components.append(f"Engine: {app.selected_engine}")
            if '400cc' in app.selected_engine:
                total_mass += 45
                total_power += 30
            elif '500cc' in app.selected_engine:
                total_mass += 55
                total_power += 40
            elif '650cc' in app.selected_engine:
                total_mass += 65
                total_power += 50
            elif '750cc' in app.selected_engine:
                total_mass += 75
                total_power += 60
            elif '1000cc' in app.selected_engine:
                total_mass += 90
                total_power += 80

        if app.selected_fuel_tank:
            components.append(f"Fuel Tank: {app.selected_fuel_tank}")
            if '15L' in app.selected_fuel_tank:
                total_mass += 12
            elif '25L' in app.selected_fuel_tank:
                total_mass += 20

        power_to_weight = total_power / total_mass if total_mass > 0 else 0

        print("REAL-TIME MOTORCYCLE SPECIFICATIONS")
        print("=" * 45)
        print(f"Type: {app.motorcycle_type}")
        print(f"Total Weight: {total_mass:.1f} kg")
        print(f"Total Power: {total_power:.1f} kW")
        print(f"Total Energy: {total_energy:.1f} kWh")
        print(f"Power-to-Weight: {power_to_weight:.3f} kW/kg")
        print(f"Components: {len(components)}/4")

        if components:
            print("\nSelected Components:")
            for comp in components:
                print(f"  - {comp}")

        # Performance indicators
        print("\nPerformance Indicators:")
        if power_to_weight > 0.15:
            print("  HIGH PERFORMANCE - Excellent acceleration and top speed")
        elif power_to_weight > 0.08:
            print("  MODERATE PERFORMANCE - Good balance of power and efficiency")
        elif power_to_weight > 0:
            print("  EFFICIENCY FOCUSED - Optimized for range and economy")
        else:
            print("  No power source selected")

        # Validation warnings
        if len(components) == 0:
            print("\nNo components selected - please select at least one component")
        elif app.motorcycle_type == "Pure EV" and not (app.selected_motor and app.selected_battery):
            print("\nPure EV requires both motor and battery")
        elif app.motorcycle_type == "Pure ICE" and not (app.selected_engine and app.selected_fuel_tank):
            print("\nPure ICE requires both engine and fuel tank")
        elif app.motorcycle_type == "Hybrid" and len(components) < 2:
            print("\nHybrid requires at least two different component types")

def build_motorcycle(btn):
    """Build the custom motorcycle."""
    with build_output:
        clear_output()
        try:
            # Validation
            components_count = sum([bool(x) for x in [app.selected_motor, app.selected_battery, app.selected_engine, app.selected_fuel_tank]])

            if components_count == 0:
                print("No components selected. Please select at least one component.")
                return

            # Type-specific validation
            if app.motorcycle_type == "Pure EV":
                if not (app.selected_motor and app.selected_battery):
                    print("Pure EV requires both motor and battery. Please select both components.")
                    return
            elif app.motorcycle_type == "Pure ICE":
                if not (app.selected_engine and app.selected_fuel_tank):
                    print("Pure ICE requires both engine and fuel tank. Please select both components.")
                    return
            elif app.motorcycle_type == "Hybrid":
                if components_count < 2:
                    print("Hybrid configuration requires at least two different component types.")
                    return

            # Build motorcycle
            motorcycle = app.build_motorcycle()

            print("CUSTOM MOTORCYCLE BUILT SUCCESSFULLY!")
            print("=" * 50)
            print(f"Name: {motorcycle.name}")
            print(f"Type: {app.motorcycle_type}")
            print(f"Total Mass: {motorcycle.mass:.1f} kg")
            print(f"Components: {len(motorcycle.child_components)}")

            # Show component list
            print("\nBuilt Components:")
            for comp in motorcycle.child_components:
                print(f"  - {comp.name} ({comp.mass:.1f} kg)")

            # Calculate total power using the known values from component selection
            total_power = 0
            if app.selected_motor:
                if '30kW' in app.selected_motor:
                    total_power += 30
                elif '50kW' in app.selected_motor:
                    total_power += 50
                elif '80kW' in app.selected_motor:
                    total_power += 80
                elif '120kW' in app.selected_motor:
                    total_power += 120
                    
            if app.selected_engine:
                if '400cc' in app.selected_engine:
                    total_power += 30
                elif '500cc' in app.selected_engine:
                    total_power += 40
                elif '650cc' in app.selected_engine:
                    total_power += 50
                elif '750cc' in app.selected_engine:
                    total_power += 60
                elif '1000cc' in app.selected_engine:
                    total_power += 80

            if total_power > 0:
                power_to_weight = total_power / motorcycle.mass
                print(f"\nPerformance Summary:")
                print(f"  - Total Power: {total_power:.1f} kW")
                print(f"  - Power-to-Weight: {power_to_weight:.3f} kW/kg")

            print("\nContinue to Step 2 to choose test tracks")

        except Exception as e:
            print(f"Error building motorcycle: {e}")
            print("Please check your component selections and try again")

# Connect event handlers
type_selector.observe(update_component_visibility, names='value')
motor_selector.observe(handle_motor_change, names='value')
battery_selector.observe(handle_battery_change, names='value')
engine_selector.observe(handle_engine_change, names='value')
fuel_selector.observe(handle_fuel_change, names='value')
build_btn.on_click(build_motorcycle)

# Initial setup
update_component_visibility()

# Display interface
display(electric_components)
display(ice_components)

print("\nSTEP 3: Monitor Real-Time Specifications")
print("=" * 50)
display(specs_display)

print("\nSTEP 4: Build Your Motorcycle")
print("=" * 50)
display(build_btn)
display(build_output)

## Step 2: Select Test Tracks

In [ ]:
#@title 🗺️ Track Selection { display-mode: "form" }

#@markdown Select which tracks to test your custom motorcycle on:
#@markdown - **Single Track**: Focused testing on specific terrain
#@markdown - **All Tracks**: Comprehensive validation across all available tracks

track_mode_selector = widgets.RadioButtons(
    options=[
        ('Test Single Track', 'single'),
        ('Test All Available Tracks', 'all')
    ],
    value='single',  # Set default value
    description='Test Mode:',
    style=style,
    layout=layout_wide
)

single_track_selector = widgets.Dropdown(
    options=[("Select a track...", None)],
    value=None,
    description='Track:',
    style=style,
    layout=layout_wide
)

track_info = widgets.Output()
confirm_tracks_btn = widgets.Button(
    description='Confirm Track Selection',
    button_style='success',
    layout=widgets.Layout(width='200px')
)

def update_track_options():
    """Update track dropdown with available tracks."""
    try:
        available_tracks = app.get_available_tracks()
        if available_tracks and available_tracks[0][1] is not None:
            options = [("Select a track...", None)] + available_tracks
            single_track_selector.options = options
        else:
            single_track_selector.options = [("No tracks available", None)]
    except Exception as e:
        print(f"Error loading tracks: {e}")

def handle_track_mode_change(change):
    """Handle track mode selection."""
    with track_info:
        clear_output()
        if change.new == 'single':
            update_track_options()
            single_track_selector.layout.display = 'block'
            print("Single Track Testing Selected")
            print("Advantages:")
            print("  - Detailed analysis of specific terrain challenges")
            print("  - Faster simulation and results")
            print("  - Focus on particular riding conditions")
            print("\nSelect a track from the dropdown below")
        elif change.new == 'all':
            single_track_selector.layout.display = 'none'
            available_tracks = app.get_available_tracks()
            if available_tracks and available_tracks[0][1] is not None:
                print(f"All Tracks Testing Selected ({len(available_tracks)} tracks)")
                print("Advantages:")
                print("  - Comprehensive performance validation")
                print("  - Statistical analysis across varied conditions")
                print("  - Complete powertrain characterization")
                print("\nAvailable tracks:")
                for i, (name, _) in enumerate(available_tracks[:8]):  # Show first 8
                    print(f"  {i+1:2d}. {name}")
                if len(available_tracks) > 8:
                    print(f"  ... and {len(available_tracks) - 8} more")
            else:
                print("No tracks found in the repository")
                print("Please ensure GPX files are available in docs/gpx_files/")

def handle_single_track_change(change):
    """Handle single track selection."""
    if change.new and track_mode_selector.value == 'single':
        with track_info:
            clear_output()
            track_name = change.new[0] if isinstance(change.new, tuple) else str(change.new)
            print(f"Selected Track: {track_name}")
            print("This track will be used for detailed performance analysis")
            print("You'll get comprehensive metrics for this specific terrain")

def confirm_track_selection(btn):
    """Confirm track selection."""
    if not app.custom_motorcycle:
        with track_info:
            clear_output()
            print("Please build a motorcycle first (Step 1)")
            print("Go back to Step 1 and click 'Build My Motorcycle'")
        return

    # Validate track mode selection
    if not track_mode_selector.value:
        with track_info:
            clear_output()
            print("Please select a test mode (Single Track or All Tracks)")
        return

    # Validate single track selection if needed
    if track_mode_selector.value == 'single':
        if not single_track_selector.value:
            with track_info:
                clear_output()
                print("Please select a specific track for single track testing")
            return

    # Store selection
    if track_mode_selector.value == 'single':
        app.selected_tracks = [single_track_selector.value]
    else:
        available_tracks = app.get_available_tracks()
        app.selected_tracks = [(name, path) for name, path in available_tracks if path is not None]

    with track_info:
        clear_output()
        print("TRACK SELECTION CONFIRMED!")
        print("=" * 40)
        print(f"Custom Motorcycle: {app.custom_motorcycle.name}")
        print(f"Total Mass: {app.custom_motorcycle.mass:.1f} kg")

        if track_mode_selector.value == 'single':
            track_name = app.selected_tracks[0][0] if isinstance(app.selected_tracks[0], tuple) else str(app.selected_tracks[0])
            print(f"Test Track: {track_name}")
            print("Mode: Single Track Analysis")
        else:
            print(f"Test Tracks: All {len(app.selected_tracks)} available")
            print("Mode: Comprehensive Validation")

        print("\nReady for simulation!")
        print("Continue to Step 3 to run the physics simulation")

# Connect event handlers
track_mode_selector.observe(handle_track_mode_change, names='value')
single_track_selector.observe(handle_single_track_change, names='value')
confirm_tracks_btn.on_click(confirm_track_selection)

# Initialize display
update_track_options()

# Check if we have a motorcycle built
if app.custom_motorcycle:
    display(track_mode_selector)
    display(single_track_selector)
    display(track_info)
    display(confirm_tracks_btn)

    # Trigger initial display
    handle_track_mode_change({'new': track_mode_selector.value})
else:
    print("Please complete Step 1 (build your motorcycle) first")
    print("Go back to Step 1 and click 'Build My Motorcycle'")

## Step 3: Run Custom Design Simulation

In [ ]:
#@title 🚀 Run Custom Design Simulation { display-mode: "form" }

#@markdown Click the button below to start the comprehensive simulation of your custom motorcycle.
#@markdown
#@markdown **Custom Design Simulation includes:**
#@markdown - Component interaction analysis
#@markdown - Real-world performance validation
#@markdown - Energy flow and efficiency calculations
#@markdown - Design optimization insights

# Create UI components
run_simulation_btn = UIComponents.create_styled_button(
    '🚀 Simulate Custom Design',
    button_style='primary',
    width='220px',
    height='40px'
)

simulation_progress, update_progress = UIComponents.create_progress_display("Simulation Progress")

def run_simulation(btn):
    """Execute simulation for custom motorcycle and tracks."""
    # Check prerequisites
    if not app.custom_motorcycle:
        update_progress("No custom motorcycle built. Complete Step 1 first.", status='error')
        return

    if not app.selected_tracks:
        update_progress("No tracks selected. Complete Step 2 first.", status='error')
        return

    update_progress("Starting custom design simulation...", progress=0.0, status='info')
    
    # Display custom motorcycle summary
    print("STARTING CUSTOM DESIGN SIMULATION")
    print("=" * 60)
    print(f"CUSTOM MOTORCYCLE: {app.custom_motorcycle.name}")
    print(f"Type: {app.motorcycle_type}")
    print(f"Total Mass: {app.custom_motorcycle.mass:.1f} kg")
    print(f"Components: {len(app.custom_motorcycle.child_components)}")

    # Show component breakdown
    print("\nCOMPONENT ANALYSIS:")
    for component in app.custom_motorcycle.child_components:
        print(f"  - {component.name} ({component.mass:.1f} kg)")

    print(f"\nTESTING ON {len(app.selected_tracks)} TRACK(S)")
    print("-" * 60)

    app.simulation_results = {}
    successful_runs = 0
    total_tracks = len(app.selected_tracks)

    # Process selected tracks
    for i, track_item in enumerate(app.selected_tracks):
        if isinstance(track_item, tuple) and len(track_item) == 2:
            track_name, track_file = track_item
        else:
            continue

        if track_file is None:
            continue

        progress = (i + 1) / total_tracks
        update_progress(f"Testing: {track_name}", progress=progress, status='info')
        print(f"\n({i+1}/{total_tracks}) Testing: {track_name}")

        try:
            result = app.simulate_motorcycle_on_track(app.custom_motorcycle, track_file, track_name)
            app.simulation_results[track_name] = result

            if result['success']:
                successful_runs += 1
                print(f"  ✅ Success - {result['total_time_s']/60:.1f} min | {result['total_distance_km']:.1f} km | {result['average_speed_kmh']:.1f} km/h")
                
                if result['energy_consumed_kWh'] > 0:
                    efficiency = result['energy_consumed_kWh'] / result['total_distance_km'] if result['total_distance_km'] > 0 else 0
                    print(f"     Energy: {result['energy_consumed_kWh']:.2f} kWh | Efficiency: {efficiency:.3f} kWh/km")
            else:
                print(f"  ❌ Failed: {result.get('error', 'Unknown error')}")

        except Exception as e:
            print(f"  ⚠️  Exception: {str(e)}")
            app.simulation_results[track_name] = {
                'track_name': track_name,
                'error': str(e),
                'success': False
            }

    # Final summary
    print("\n" + "=" * 60)
    update_progress(f"Simulation Complete! {successful_runs}/{total_tracks} tracks successful", 
                   progress=1.0, 
                   status='success' if successful_runs > 0 else 'warning')
    
    if successful_runs > 0:
        # Calculate summary metrics
        successful_results = [r for r in app.simulation_results.values() if r.get('success')]
        total_distance = sum(r['total_distance_km'] for r in successful_results)
        total_time = sum(r['total_time_s'] for r in successful_results)
        total_energy = sum(r['energy_consumed_kWh'] for r in successful_results)
        avg_speed = (total_distance / (total_time / 3600)) if total_time > 0 else 0

        print(f"\nCUSTOM DESIGN PERFORMANCE SUMMARY:")
        print(f"  Total Distance Tested: {total_distance:.1f} km")
        print(f"  Total Test Time: {total_time/3600:.1f} hours")
        print(f"  Average Speed: {avg_speed:.1f} km/h")

        if total_energy > 0:
            print(f"  Total Energy Consumed: {total_energy:.2f} kWh")
            efficiency = total_energy / total_distance if total_distance > 0 else 0
            print(f"  Overall Efficiency: {efficiency:.3f} kWh/km")

        print("\n📊 Continue to Step 4 to analyze results and export design data")
    else:
        print("\n❌ No successful simulations completed.")
        print("Please check your component configuration and try again.")

run_simulation_btn.on_click(run_simulation)

# Only show if prerequisites are met
if app.custom_motorcycle and app.selected_tracks:
    display(run_simulation_btn)
    display(simulation_progress)
else:
    missing = []
    if not app.custom_motorcycle:
        missing.append("custom motorcycle (Step 1)")
    if not app.selected_tracks:
        missing.append("test tracks (Step 2)")

    print(f"⚠️  Complete the following first: {', '.join(missing)}")
    for item in missing:
        if "motorcycle" in item:
            print("  📍 Go to Step 1 and build your motorcycle")
        if "tracks" in item:
            print("  📍 Go to Step 2 and select test tracks")

## Step 4: Design Analysis & Export

In [ ]:
#@title 📊 Custom Design Analysis & Export { display-mode: "form" }

#@markdown This section provides comprehensive analysis of your custom motorcycle design and professional export capabilities.
#@markdown **Perfect for engineering analysis** - includes component specifications, performance metrics, and design insights.

view_design_analysis_btn = UIComponents.create_styled_button(
    'View Design Analysis',
    button_style='info',
    width='200px'
)

export_design_btn = UIComponents.create_styled_button(
    'Export Design Data',
    button_style='success',
    width='200px'
)

analysis_display = widgets.Output()
export_status = widgets.Output()

def view_design_analysis(btn):
    """Display comprehensive custom design analysis."""
    if not app.simulation_results:
        with analysis_display:
            clear_output()
            print("❌ No simulation results available. Run simulation first.")
        return

    if not app.custom_motorcycle:
        with analysis_display:
            clear_output()
            print("❌ No custom motorcycle found. Build motorcycle first.")
        return

    with analysis_display:
        clear_output()

        print("COMPREHENSIVE CUSTOM DESIGN ANALYSIS")
        print("=" * 80)

        # Custom motorcycle specifications
        print(f"CUSTOM MOTORCYCLE: {app.custom_motorcycle.name}")
        print(f"Powertrain Type: {app.motorcycle_type}")
        print(f"Total Mass: {app.custom_motorcycle.mass:.1f} kg")
        print(f"Custom Components: {len(app.custom_motorcycle.child_components)}")

        # Detailed component analysis
        print(f"\nCOMPONENT SPECIFICATION ANALYSIS:")
        print("-" * 80)

        # Use component summary cards from UIComponents
        for component in app.custom_motorcycle.child_components:
            specs = {
                'Mass': component.mass,
                'Type': type(component).__name__
            }
            
            # Extract power from component name
            if 'kW' in component.name:
                try:
                    power_str = component.name.split('kW')[0].split()[-1]
                    power_kw = float(power_str)
                    specs['Max Power'] = f"{power_kw} kW"
                    specs['Power/Weight'] = f"{(power_kw / component.mass):.3f} kW/kg"
                except:
                    pass

            if hasattr(component, 'remaining_energy_capacity'):
                energy_kwh = component.remaining_energy_capacity / 3.6e6
                specs['Energy Capacity'] = f"{energy_kwh:.1f} kWh"
                specs['Energy Density'] = f"{(energy_kwh / component.mass * 1000):.0f} Wh/kg"

            # Display component card
            display(HTML(UIComponents.create_component_summary_card(component.name, specs)))

        # Performance validation results
        successful_results = [r for r in app.simulation_results.values() if r.get('success')]
        
        if successful_results:
            print(f"\nPERFORMANCE VALIDATION RESULTS")
            print("-" * 100)
            print(f"{'Track Name':<40} {'Distance':<10} {'Time':<8} {'Speed':<10} {'Energy':<10} {'Efficiency':<15}")
            print(f"{'':40} {'(km)':<10} {'(min)':<8} {'(km/h)':<10} {'(kWh)':<10} {'(kWh/km)':<15}")
            print("-" * 100)

            for result in successful_results:
                track_name = result['track_name'][:39]
                dist = result['total_distance_km']
                time_min = result['total_time_s'] / 60
                speed = result['average_speed_kmh']
                energy = result['energy_consumed_kWh']
                efficiency = energy / dist if dist > 0 and energy > 0 else 0

                print(f"{track_name:<40} {dist:<10.1f} {time_min:<8.1f} {speed:<10.1f} {energy:<10.2f} {efficiency:<15.4f}")

            print("-" * 100)

            # Statistical analysis
            speeds = [r['average_speed_kmh'] for r in successful_results]
            print(f"\nSTATISTICAL PERFORMANCE ANALYSIS:")
            print(f"  - Mean Speed: {np.mean(speeds):.1f} km/h")
            print(f"  - Speed Range: {np.min(speeds):.1f} - {np.max(speeds):.1f} km/h")
            print(f"  - Speed Consistency: {(1 - np.std(speeds)/np.mean(speeds))*100:.1f}%")
        else:
            print("\n❌ No successful simulation results to analyze")

def export_design_data(btn):
    """Export comprehensive custom design data."""
    if not app.simulation_results:
        with export_status:
            clear_output()
            print("❌ No results to export. Run simulation first.")
        return

    if not app.custom_motorcycle:
        with export_status:
            clear_output()
            print("❌ No custom motorcycle found. Build motorcycle first.")
        return

    try:
        with export_status:
            clear_output()
            print("📦 Preparing comprehensive custom design export...")

        # Prepare component selections
        component_selections = {
            'selected_motor': app.selected_motor,
            'selected_battery': app.selected_battery,
            'selected_engine': app.selected_engine,
            'selected_fuel_tank': app.selected_fuel_tank
        }

        # Use SimulationExporter for standardized export
        filename = SimulationExporter.export_custom_design(
            app.custom_motorcycle,
            app.motorcycle_type,
            app.simulation_results,
            component_selections
        )

        with export_status:
            clear_output()
            print("✅ CUSTOM DESIGN EXPORT COMPLETED SUCCESSFULLY!")
            print("=" * 60)
            print(f"📄 Filename: {filename}")

            file_size = os.path.getsize(filename) / 1024  # KB
            print(f"💾 File Size: {file_size:.1f} KB")

            print("\n📦 Export Contains:")
            print("  - Custom design specifications and component selections")
            print("  - Detailed component specifications")
            print(f"  - {len(app.simulation_results)} track simulation results")
            print("  - Performance statistics and metrics")
            print("  - Design optimization insights")

            successful_results = [r for r in app.simulation_results.values() if r.get('success')]
            if successful_results:
                print(f"\n📊 Key Metrics Exported:")
                print(f"  - Validation Success: {len(successful_results)}/{len(app.simulation_results)} tracks")
                avg_speed = np.mean([r['average_speed_kmh'] for r in successful_results])
                total_dist = sum([r['total_distance_km'] for r in successful_results])
                print(f"  - Average Performance: {avg_speed:.1f} km/h over {total_dist:.1f} km")

            print("\n🎯 Ready for comprehensive engineering analysis!")

    except Exception as e:
        with export_status:
            clear_output()
            print(f"❌ Export failed: {str(e)}")
            print("Please check file permissions and available disk space")

# Connect button handlers
view_design_analysis_btn.on_click(view_design_analysis)
export_design_btn.on_click(export_design_data)

# Only show if simulation has been run
if app.simulation_results:
    display(widgets.HBox([view_design_analysis_btn, export_design_btn]))
    display(analysis_display)
    display(export_status)
else:
    print("⚠️  Run simulation first (Step 3) to analyze and export your custom design")
    print("📍 Complete Steps 1-3 to unlock comprehensive design analysis")